## Лабораторная №5. Обработка текстовых данных из открытых источников

[отчет](https://docs.google.com/document/d/1adUHKbePxO45GBTsv93WGicj1wH8QDkbG5QIR0K2WG0/edit?usp=sharing)

In [ ]:
!pip install pymorphy2

In [ ]:
import pandas as pd
import pymorphy2

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords

import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
PATH = 'https://raw.githubusercontent.com/WebOfRussia/financial-news-sentiment/main/data/data.tsv'
data = pd.read_csv(PATH, sep = '\t')

data.head()

,title,score,link,summary,published,tickers
0,Электромобильный стартап Arrival экс-главы Yot...,-0.583333,https://www.rbc.ru/technology_and_media/12/05/...,"Британский электромобильный стартап Arrival, к...","Thu, 12 May 2022 05:10:01 +0300",['ARVL']
1,Экс-глава НМТП рассказал о «напряженных отноше...,-0.314286,https://www.rbc.ru/society/16/11/2020/5fb2709d...,Экс-председатель совета директоров Новороссийс...,"Fri, 20 May 2022 19:13:18 +0300",['NMTP']
2,Шрёдер отклонил предложение войти в совет дире...,-0.333333,https://www.rbc.ru/business/20/05/2022/628772b...,Его кандидатуру выдвинули в начале февраля. Ка...,"Tue, 24 May 2022 22:12:05 +0300",['GAZP']
3,Шельф берут в разработку // Генподрядчиком «Га...,0.700000,https://www.kommersant.ru/doc/5482398,"Как стало известно “Ъ”, «Аврора» Андрея Патруш...","Fri, 29 Jul 2022 00:28:00 +0300",['GAZP']
4,"Чистый убыток ""Юнипро"" в 1 полугодии 2022 года...",-0.611111,https://www.finam.ru/analysis/newsitem/chistyi...,"Чистый убыток ""Юнипро"" в 1 полугодии 2022 года...","Thu, 28 Jul 2022 12:43:00 +0300",['UPRO']


In [ ]:
morph = pymorphy2.MorphAnalyzer()

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def lemmatize_tokenize_text(text: str) -> list:
  '''
  Токенизация и лемматизация описания новости

  Returns: обработанная строка со словами в нормальной форме
  '''
  text = str(text)
  text = re.sub(r'[^\w\s]', '', text)
  tokens = word_tokenize(text)
  tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]
  lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
  filtered_list = [s for s in lemmatized_tokens if not s.isdigit()]
  filtered_list_2 = [word for word in filtered_list if len(word) >= 3]

  return filtered_list_2


def replace_scores(x: float) -> int:
  '''
  Заменяем значения в столбце 'score', чтобы было удобнее определять тональность

  Returns: целое число, характеризующее тональность новости
  '''
  if x > 0:
      return 1
  elif x < 0:
      return -1
  else:
      return 0

In [ ]:
data_lemma = data.copy()
data_lemma['summary'] = data_lemma['summary'].apply(lambda x: lemmatize_tokenize_text(x))
data_lemma['summary'] = data_lemma['summary'].apply(lambda x: ' '.join(x))

data_lemma['summary']

0      британский электромобильный стартап arrival ко...
1      экспредседатель совет директор новороссийский ...
2      кандидатура выдвинуть начало февраль писать fo...
3      стать известно аврора андрей патрушев новый на...
4      чистый убыток юнипро полугодие год рсб состави...
                             ...                        
527    куйбышевазот зафиксировать рост выработка осно...
528    сегодняшний день газпром прекратить поставка г...
529    газпром опубликовать часть документ портовый п...
530    газпром нефть испытать холмогорский месторожде...
531    аэрофлот август открывать прямой регулярный ре...
Name: summary, Length: 532, dtype: object

In [ ]:
data_lemma['score'] = data_lemma['score'].apply(replace_scores)
data_lemma.head()

,title,score,link,summary,published,tickers
0,Электромобильный стартап Arrival экс-главы Yot...,-1,https://www.rbc.ru/technology_and_media/12/05/...,британский электромобильный стартап arrival ко...,"Thu, 12 May 2022 05:10:01 +0300",['ARVL']
1,Экс-глава НМТП рассказал о «напряженных отноше...,-1,https://www.rbc.ru/society/16/11/2020/5fb2709d...,экспредседатель совет директор новороссийский ...,"Fri, 20 May 2022 19:13:18 +0300",['NMTP']
2,Шрёдер отклонил предложение войти в совет дире...,-1,https://www.rbc.ru/business/20/05/2022/628772b...,кандидатура выдвинуть начало февраль писать fo...,"Tue, 24 May 2022 22:12:05 +0300",['GAZP']
3,Шельф берут в разработку // Генподрядчиком «Га...,1,https://www.kommersant.ru/doc/5482398,стать известно аврора андрей патрушев новый на...,"Fri, 29 Jul 2022 00:28:00 +0300",['GAZP']
4,"Чистый убыток ""Юнипро"" в 1 полугодии 2022 года...",-1,https://www.finam.ru/analysis/newsitem/chistyi...,чистый убыток юнипро полугодие год рсб состави...,"Thu, 28 Jul 2022 12:43:00 +0300",['UPRO']


In [ ]:
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# BoW и классификация с помощью логистической регрессии
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(data_lemma['summary'])
y = data_lemma['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)

accuracy_lin_bow = accuracy_score(y_test, y_pred_log)


# BoW и классификация с помощью наивного байеса
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_lemma['summary'])
y = data_lemma['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=22)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred_nb = nb_classifier.predict(X_test)

accuracy_nb_bow = accuracy_score(y_test, y_pred_nb)

In [ ]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

          -1       0.60      0.67      0.63        27
           0       0.00      0.00      0.00         1
           1       0.58      0.54      0.56        26

    accuracy                           0.59        54
   macro avg       0.39      0.40      0.40        54
weighted avg       0.58      0.59      0.59        54



In [ ]:
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

          -1       0.72      0.78      0.75        27
           0       0.00      0.00      0.00         1
           1       0.72      0.69      0.71        26

    accuracy                           0.72        54
   macro avg       0.48      0.49      0.49        54
weighted avg       0.71      0.72      0.71        54



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# TF-IDF и классификация с помощью логистической регрессии
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(data_lemma['summary'])
y = data_lemma['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred_nb = nb_classifier.predict(X_test)

accuracy_lin_idf = accuracy_score(y_test, y_pred_nb)



# TF-IDF и классификация с помощью наивного байеса
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(data_lemma['summary'])
y = data_lemma['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=22)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred_log = nb_classifier.predict(X_test)

accuracy_nb_idf = accuracy_score(y_test, y_pred_log)

In [ ]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

          -1       0.66      0.93      0.77        27
           0       0.00      0.00      0.00         1
           1       0.81      0.50      0.62        26

    accuracy                           0.70        54
   macro avg       0.49      0.48      0.46        54
weighted avg       0.72      0.70      0.68        54



In [ ]:
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

          -1       0.53      0.63      0.58        27
           0       0.00      0.00      0.00         1
           1       0.55      0.46      0.50        26

    accuracy                           0.54        54
   macro avg       0.36      0.36      0.36        54
weighted avg       0.53      0.54      0.53        54



In [ ]:
print(f"BoW ------->    NB: {round(accuracy_nb_bow, 3)};    LOGISTIC: {round(accuracy_lin_bow, 3)}" + "\n" +
      f"TF-IDF ---->    NB: {round(accuracy_nb_idf, 3)};    LOGISTIC: {round(accuracy_lin_idf, 3)}")

BoW ------->    NB: 0.722;    LOGISTIC: 0.648
TF-IDF ---->    NB: 0.704;    LOGISTIC: 0.981


Таким образом, лучшие показатели точности предсказания для логистической регрессии достигаются, если не использовать параметр random_state. В то время как для наивный байес наоборот, повышает свои показатели с этим параметром.